In [1]:
!pip install tensorflowjs

     |████████████████████████████████| 64 kB 2.6 MB/s 


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import tensorflow as tf
import zipfile as zip
import glob
from sklearn.model_selection import train_test_split
import os as os
import shutil

In [4]:
zip_path = "/content/drive/MyDrive/Train-Test/"

In [5]:
with zip.ZipFile(zip_path + 'train.zip', 'r') as zipObj:
   zipObj.extractall()
  
with zip.ZipFile(zip_path + 'test1.zip', 'r') as zipObj:
   zipObj.extractall()

In [6]:
test_imgpath = "/content/test1/"
train_imgpath = "/content/train/"
val_imgpath = "/content/val/"

In [7]:
train_list = glob.glob(os.path.join(train_imgpath,'*.jpg'))
test_list = glob.glob(os.path.join(test_imgpath, '*.jpg'))

In [8]:
print(len(train_list),len(test_list))

25000 12500


In [9]:
train_list, val_list = train_test_split(train_list, test_size=0.2,shuffle=True)

In [10]:
print(len(train_list),len(val_list))

20000 5000


In [11]:
os.mkdir(val_imgpath)

In [13]:
def copy_to_val_folder(val_list):
  val_path = "/content/val"
  for images in val_list:
    shutil.move(images,val_path)

In [14]:
def delete_train_test_folder(folder):
  for images in os.listdir("/content/".format(folder)):
      os.remove(os.path.join("/content/{}".format(folder) + "/" + images))
  os.removedirs("/content/".format(folder))

In [15]:
def empty_val_folder():
  for images in os.listdir(val_path):
    os.remove(os.path.join(val_path + "/" + images))

In [16]:
copy_to_val_folder(val_list)

In [17]:
def align_train_data():
  os.mkdir(train_imgpath + "dog")
  os.mkdir(train_imgpath + "cat")

  for image in os.listdir(train_imgpath):
    if image.split(".")[0] == "dog":
      shutil.move(train_imgpath + image,train_imgpath + "dog")
    else:
      shutil.move(train_imgpath + image,train_imgpath + "cat")

align_train_data()

In [18]:
def align_val_data():
  os.mkdir(val_imgpath + "dog")
  os.mkdir(val_imgpath + "cat")

  for image in os.listdir(val_imgpath):
    if image.split(".")[0] == "dog":
      shutil.move(val_imgpath + image,val_imgpath + "dog")
    else:
      shutil.move(val_imgpath + image,val_imgpath + "cat")

align_val_data()

In [19]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                                  horizontal_flip=True,
                                                                  validation_split=0.2)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

In [32]:
train_images = train_generator.flow_from_directory(
    directory = train_imgpath,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=100,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_directory(
    directory=val_imgpath,
    target_size=(224, 224),
    color_mode='rgb',
    class_mode='binary',
    batch_size=100,
    shuffle=True,
    seed=42,
    subset='validation'
)


Found 16001 images belonging to 2 classes.
Found 999 images belonging to 2 classes.


In [33]:
 (20000 + 5000) - (16001 + 999) 

8000

In [54]:
model = tf.keras.Sequential([
                             
      tf.keras.layers.Conv2D(16,(3,3),(2,2),activation="relu",input_shape=(224,224,3)),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.MaxPooling2D((2,2),strides=2),
      tf.keras.layers.Dropout(.25),

      tf.keras.layers.Conv2D(32,(5,5),(2,2),activation="relu"),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.MaxPooling2D((2,2),strides=2),
      tf.keras.layers.Dropout(.25),

      tf.keras.layers.Conv2D(64,(5,5),(2,2),activation="relu"),
      tf.keras.layers.BatchNormalization(),
      tf.keras.layers.MaxPooling2D((2,2),strides=2),
      tf.keras.layers.Dropout(.25),

      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(100,activation="relu"),
      tf.keras.layers.Dropout(.25),
      tf.keras.layers.Dense(50,activation="relu"),
      tf.keras.layers.Dropout(.25),
      tf.keras.layers.Dense(20,activation="relu"),
      tf.keras.layers.Dropout(.25),
      tf.keras.layers.Dense(1,activation="softmax")

])


In [55]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [56]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 111, 111, 16)      448       
_________________________________________________________________
batch_normalization_12 (Batc (None, 111, 111, 16)      64        
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 55, 55, 16)        0         
_________________________________________________________________
dropout_21 (Dropout)         (None, 55, 55, 16)        0         
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 26, 26, 32)        12832     
_________________________________________________________________
batch_normalization_13 (Batc (None, 26, 26, 32)        128       
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 13, 13, 32)       

In [52]:
imagess = next(train_images)

In [53]:
imagess[1].shape

(100,)

In [ ]:
history = model.fit(
    train_images,
    validation_data=val_images,
    epoch=15
)